# Guide for a better C++

## Todo

- [x] Example with string
- [x] Example with matrices
- [x] Eager Evaluation
- [x] Return value optimization
- [x] Cost of virtual functions
- [x] Virtualizing non member functions
- [x] Requiring or prohibiting heap-based objects
- [x] Multiple Dispatching
- [x] Finish More Effective C++
- [ ] Difference between const and constexpr with assmebler explanation
- [ ] Check Combine C++ and C (for personnal purpose not for the guide)
- [ ] Check C++17 specs
- [ ] Check that all snippet of codes compile


## Topics to deal with

### C++ Tips

#### <ins>Lazy evaluation:</ins>

**Principle: Do not evaluate an expression until the results is required.**

This method might save computations whose results are not used.
Here are 4 example of lazy evaluation application areas:

- **Reference counting:** Count references to objects instead of always copying data.
- **Distinguish Reads from Write:** When using reference counting, create new copie of data only when using *operator[]* to write.
- **Lazy Fetching:** When dealing with large objects, read no data from object until a specific attribute is asked for Read/Write.
- **Lazy Evaluation:** When making operations on structure, store the operation in a structure and defer computation until result is required.

##### <ins>Example 1:</ins>

In [9]:
std::string s1 = "Hello";
std::string s2 = s1; //Calls string copy ctor which performs an allocation 
                //then a strcopy to copy data
                //(s1 and s2 have their own copy of the value)

printf("%p\n", s1.data());
printf("%p\n", s2.data());

0x7f113d9ae0f8
0x7f113d9ae118


This is how data is stored, however it is not needed to have data duplicated until s2 is modified.

<div style="text-align:center"><img src="https://i.imgur.com/zxaUBIm.png" /></div>

One method is to implement a struct nested in String that holds a *referenceCounter* and avoid data duplication until it is really needed.
Here is a graphical representation of this model:


<div style="text-align:center"><img src="https://i.imgur.com/Ov7zKM4.png" /></div>

Snippet of code representing this model.

In [13]:
class String
{
    struct StringValue {
        //Init data and strcopy initValue inside
        //Set refCount to 1
        StringValue(const char* initValue);
        
        //Delete data
        ~StringValue();

        size_t m_sRefCount;
        char* m_pData;        
    };
    
    
    String(const char* data="")
    : m_pValue(new StringValue(data))
    {    
    }
    
    //Destroy StringValue
    ~String()
    {
        delete m_pValue;
    }
    
    //Copy constructor
    String(const String& rhs)
    {
        if (m_pValue == rhs.m_pValue)    return *this;
        
        //By doing this, we postpone data duplication
        // data evulation consisting on: creating a new char[] and calling strcopy
        if (--m_pValue->m_sRefCount == 0)    delete m_pValue;
        
        m_pValue = rhs.m_pValue;
        m_pValue->m_sRefCount++;
        return *this;
    }
    
    //Evaluation of duplication operation
    void EvaluateDuplication()
    {
        //Decrease reference counter
        m_pValue->m_sRefCount--;    
        
        //Creates its own StringValue
        m_pValue = new StringValue(m_pValue->m_pData);
    }
    
    //Writing operation: data duplication must be evaluated
    void ToLowerCase()
    {
        if (m_pValue->m_sRefCount > 1)
            EvaluateDuplication();
            
        //... LowerCase operation
    }
    
    StringValue* m_pValue;
};

In [14]:
String S1 = "Hello";
String S2 = S1;

//show data addr

input_line_20:2:9: error: calling a private constructor of class '__cling_N513::String'
 String S1 = "Hello";
        ^
input_line_19:15:5: note: implicitly declared private here
    String(const char* data="")
    ^
input_line_20:2:14: error: temporary of type '__cling_N513::String' has private destructor
 String S1 = "Hello";
             ^
input_line_19:21:5: note: implicitly declared private here
    ~String()
    ^
input_line_20:2:9: error: variable of type '__cling_N513::String' has private destructor
 String S1 = "Hello";
        ^
input_line_19:21:5: note: implicitly declared private here
    ~String()
    ^
input_line_20:3:13: error: calling a private constructor of class '__cling_N513::String'
String S2 = S1;
            ^
input_line_19:27:5: note: implicitly declared private here
    String(const String& rhs)
    ^
input_line_20:3:8: error: variable of type '__cling_N513::String' has private destructor
String S2 = S1;
       ^
input_line_19:21:5: note: implicitly declared pr

Interpreter Error: 

In [9]:
Test t;

Hello, I am a Test instance !
